## Imports

In [ ]:
import sys
sys.path.append('../../')
import numpy as np
import torch
from torchvision import transforms
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image, ImageOps
import matplotlib.patches as patches
from scipy.spatial.transform import Rotation
import pandas as pd
from scipy.spatial import distance
import time
import os
import math
import scipy.io as sio
from utils.renderer import Renderer
from utils.image_operations import expand_bbox_rectangle
from utils.pose_operations import get_pose
from img2pose import img2poseModel
from model_loader import load_model

np.set_printoptions(suppress=True)

## Declare useful functions

In [ ]:
def render_plot(img, poses, bboxes):
    (w, h) = img.size
    image_intrinsics = np.array([[w + h, 0, w // 2], [0, w + h, h // 2], [0, 0, 1]])
    
    trans_vertices = renderer.transform_vertices(img, poses)
    img = renderer.render(img, trans_vertices, alpha=1)    

    plt.figure(figsize=(8, 8))     
    
    for bbox in bboxes:
        plt.gca().add_patch(patches.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1],linewidth=3,edgecolor='b',facecolor='none'))            
    
    plt.imshow(img)        
    plt.show()

## Create the renderer for visualization

In [ ]:
renderer = Renderer(
    vertices_path="../../pose_references/vertices_trans.npy", 
    triangles_path="../../pose_references/triangles.npy"
)

threed_points = np.load('../../pose_references/reference_3d_68_points_trans.npy')

## Load model weights and pose mean and std deviation
To test other models, change MODEL_PATH along the the POSE_MEAN and POSE_STDDEV used for training

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

DEPTH = 18
MAX_SIZE = 1400
MIN_SIZE = 600

POSE_MEAN = "../../models/WIDER_train_pose_mean_v1.npy"
POSE_STDDEV = "../../models/WIDER_train_pose_stddev_v1.npy"
MODEL_PATH = "../../models/img2pose_v1.pth"

pose_mean = np.load(POSE_MEAN)
pose_stddev = np.load(POSE_STDDEV)

img2pose_model = img2poseModel(
    DEPTH, MIN_SIZE, MAX_SIZE, 
    pose_mean=pose_mean, pose_stddev=pose_stddev,
    threed_68_points=threed_points,
)
load_model(img2pose_model.fpn_model, MODEL_PATH, cpu_mode=str(img2pose_model.device) == "cpu", model_only=True)
img2pose_model.evaluate()

In [ ]:
import cv2
import os

# Open the video file

video = cv2.VideoCapture('/content/img2pose/data/sample.mp4')

# Get the frames per second (fps) of the video
fps = int(video.get(cv2.CAP_PROP_FPS))

# Initialize frame counter
frame_count = 0

# Create a directory to store the frames
if not os.path.exists('/content/img2pose/results/frames'):
    os.makedirs('/content/img2pose/results/frames')

# Loop through the frames of the video
while True:
    # Read the next frame
    ret, frame = video.read()

    # If there are no more frames, break out of the loop
    if not ret:
        break

    # Save the current frame as an image file in the 'frames' folder
    cv2.imwrite(f'/content/img2pose/results/frames/frame{frame_count}.jpg', frame)

    # Increment the frame counter
    frame_count += 1

    # Wait for a short time to simulate the video playback speed
    cv2.waitKey(int(1000/fps))

# Release the video file
video.release()

In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as patches
 

# change to a folder with images, or another list containing image paths
images_path = "/content/img2pose/results/frames"

# Create a directory to store the frames
if not os.path.exists('/content/img2pose/results/maskedframes'):
    os.makedirs('/content/img2pose/results/maskedframes')

threshold = 0.9

if os.path.isfile(images_path):
    img_paths = pd.read_csv(images_path, delimiter=" ", header=None)
    img_paths = np.asarray(img_paths).squeeze()
else:
    img_paths = [os.path.join(images_path, img_path) for img_path in os.listdir(images_path) if img_path.endswith(".jpg")]

for img_path in tqdm(img_paths):
    img = Image.open(img_path).convert("RGB")
    
    image_name = os.path.split(img_path)[1]
    
    (w, h) = img.size
    image_intrinsics = np.array([[w + h, 0, w // 2], [0, w + h, h // 2], [0, 0, 1]])
            
    res = img2pose_model.predict([transform(img)])[0]

    all_bboxes = res["boxes"].cpu().numpy().astype('float')

    poses = []
    bboxes = []
    for i in range(len(all_bboxes)):
        if res["scores"][i] > threshold:
            bbox = all_bboxes[i]
            pose_pred = res["dofs"].cpu().numpy()[i].astype('float')
            pose_pred = pose_pred.squeeze()

            poses.append(pose_pred)  
            bboxes.append(bbox)

    # Assuming you have already initialized the renderer and loaded the image
    trans_vertices = renderer.transform_vertices(img, poses)
    img = renderer.render(img, trans_vertices, alpha=1)

    # Create a new plot
    fig, ax = plt.subplots(figsize=(8, 8))

    # Display the image
    ax.imshow(img)

    # Add bounding boxes to the plot (assuming you have already defined the bboxes)
    for bbox in bboxes:
        ax.add_patch(patches.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1],
                                        linewidth=3, edgecolor='b', facecolor='none'))

    # Save the plot to a file
    plt.savefig('/content/img2pose/results/maskedframes/'+image_name, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
import cv2
import os

# Path to the directory containing the frames
frames_dir = '/content/img2pose/results/maskedframes'

# Get a list of all the frame filenames
frame_files = sorted(os.listdir(frames_dir))

# Load the first frame to get the frame size
frame = cv2.imread(os.path.join(frames_dir, frame_files[0]))
height, width, channels = frame.shape

# Create the video writer object
video_writer = cv2.VideoWriter('/content/img2pose/results/masked_video_people.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 25, (width, height))

# Loop over all the frames and add them to the video
for i in range(1, len(frame_files)):
    filename =frames_dir+'/frame'+str(i)+'.jpg'
    frame = cv2.imread(filename)
    video_writer.write(frame)

# Release the video writer and close the video file
video_writer.release()